In [1]:
import os
import io
import json
import pymorphy2
from gensim import corpora
from collections import Counter

islamcivilru


In [2]:
# -*- coding: utf-8 -*-
POSOpt = ["NOUN",	#имя существительное	хомяк
        "ADJF",	#имя прилагательное (полное)	хороший
        "ADJS",	#имя прилагательное (краткое)	хорош
        "VERB",	#глагол (личная форма)	говорю, говорит, говорил
        "INFN",	#глагол (инфинитив)	говорить, сказать
        "PRTF",	#причастие (полное)	прочитавший, прочитанная
        "PRTS"] #причастие (краткое)	прочитана
morph = pymorphy2.MorphAnalyzer()

In [3]:
def POSFilter(word):
    p = morph.parse(word)
    keyPos = p[0].tag.POS
    if keyPos in POSOpt:
        return True;
    return False

In [4]:
#contrastStats = getContrastCorpus()
statsCorpusDir = 'C:\\NIVC\\Nivc_stats_corpus\\unary_comm'
BigARTMCorpusDir = 'C:\\NIVC\\Nivc_GenSim_corpus\\unary_comm\\'
mainTree = os.walk(statsCorpusDir)
allStatsFilesRus = []
texts = []
i = 0
corpus = []
dictionary = corpora.Dictionary()
err = False
for d in mainTree:
    for subd in d[1][2:3]:
        print(subd)
        newPath = d[0] + "\\" + subd
        subTree = os.walk(newPath)
        i = 1
        for f in subTree:
            for subf in f[2]:
                filePath = f[0] + "\\" + subf
                #print(subf)
                if "_Stat" in subf and "_rus" in subf:
                    allStatsFilesRus.append([filePath,subf])

        if not os.path.exists(BigARTMCorpusDir+subd+"\\"+"mm_pos"):
            os.makedirs(BigARTMCorpusDir+subd+"\\"+"mm_pos", mode=0o777)
        
        for file in allStatsFilesRus:
            content = ""
            contentDict = {}
            with io.open(file[0],'r',encoding='utf8') as inputStat:
                try:
                    content = inputStat.read()
                except Exception as e:
                    print("Error",file)
                    continue
                inputStat.close()
            try:
                contentDict = json.loads(content)
            except Exception as e:
                print(file,content)
            vowpalWabbitStr = ""
            textWords = [unicode(key) for key in contentDict["Frequencies"] if (unicode(key) != unicode("") and unicode(key) != unicode(" ")) and POSFilter(key)]
            newBow = dictionary.doc2bow(textWords,True) 
            for key in textWords:
                id = dictionary.token2id[unicode(key)]
                ind = [i for i, x in enumerate(newBow) if isinstance(x, tuple) and x[0] == id]
                if ind != []:
                    newBowList = list(newBow[ind[0]])
                    newBowList[1] = contentDict["Frequencies"][key]
                    newBow[ind[0]] = tuple(newBowList)
                else:
                    err = True
                    print "Error"
                    break
            if err: break
            corpus.append(newBow)
        if err: break
    if err: break

islamcivilru


In [5]:
print dictionary

Dictionary(24774 unique tokens: [u'\u0412\u041e\u041b\u042c\u041d\u041e\u041e\u0422\u041f\u0423\u0429\u0415\u041d\u041d\u042b\u0419', u'\u0421\u0410\u0425\u0423\u0420\u041e\u041c', u'\u041a\u0410\u0411\u0410\u041a', u'\u0410\u041d\u041d\u0418', u'\u0420\u042b\u0426\u0410\u0420\u0421\u041a\u0418\u0419']...)


In [6]:
dictionary.save(BigARTMCorpusDir+subd+"\\"+"mm_pos\\"+subd+".dict")

In [8]:
corpora.MmCorpus.serialize(BigARTMCorpusDir+subd+"\\"+"mm_pos\\"+subd+'.mm', corpus)